### Initial Setup

In [2]:
# imports
import numpy as np
import pandas as pd
import requests
import spotipy
import librosa
import json
import re
import urllib.request
import langdetect as ld

from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup

In [3]:
# spotify api creds
SPOTIFY_CLIENT_ID = ''
SPOTIFY_CLIENT_SECRET = ''

In [4]:
# function to get spotify token
def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    return auth_response_data['access_token']

In [5]:
# track details function
def get_track_details(spotify_id, token):
    sp = spotipy.Spotify(auth=token)
    track = sp.track(spotify_id)
    return track['name'], track['artists'][0]['name']

--- 

### Lyrics - Web Scraping Approach (Hit or Miss)

In [ ]:
 
# function 1 to get the lyrics from spotify id
def get_lyrics_by_spotify_id(spotify_id): 

    spotify_token = get_spotify_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    song_title, artist = get_track_details(spotify_id, spotify_token)

    artist = artist.lower() 
    song_title = song_title.lower()

    # remove all except alphanumeric characters from artist and song_title 
    artist = re.sub('[^A-Za-z0-9]+', "", artist) 
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title)

    # remove starting 'the' from artist e.g. the who -> who
    if artist.startswith("the"):     
        artist = artist[3:] 
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html"

     
    try: 
        content = urllib.request.urlopen(url).read() 
        soup = BeautifulSoup(content, 'html.parser') 
        lyrics = str(soup) 
        # lyrics lies between up_partition and down_partition 
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->' 
        down_partition = '<!-- MxM banner -->' 
        lyrics = lyrics.split(up_partition)[1] 
        lyrics = lyrics.split(down_partition)[0] 
        lyrics = lyrics.replace('<br>','').replace('</br>','').replace('</div>','').replace('<br/>','').strip()
        
        return lyrics     
    except Exception as e: 
        return "Exception occurred \n" +str(e)

### Lyrics - Free API (Adds jargon in the first line)

In [13]:
# function 2 to get the lyrics from spotify id
def get_lyrics_by_spotify_id(spotify_id): 

    spotify_token = get_spotify_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    song_title, artist = get_track_details(spotify_id, spotify_token)

    artist = str(artist) 
    song_title = str(song_title)

    url = 'https://api.lyrics.ovh/v1/' + artist + '/' + song_title

    # fetch lyrics
    response = requests.get(url)
    json_data = json.loads(response.content)
    lyrics = json_data['lyrics']

    return lyrics   

In [14]:
# example usage
spotify_id = 'https://open.spotify.com/track/0TL0LFcwIBF5eX7arDIKxY?si=61eef086ea274c4a'
lyrics = get_lyrics_by_spotify_id(spotify_id)

print(lyrics.lower())

paroles de la chanson husn par anuv jain
dekho, dekho, kaisi baatein yahan ki
hain saath par, hain saath naa bhi
kya itni aasaan hai?
dekho, dekho, jaise mere iraade
waise kahaan tere yahan the
haan, kitni naadan main

mere husn ke ilaawa kabhi dil bhi maang lo naa
haaye, pal mei main pighal jaaun, haan
ab aisa naa kro ki dil jud na paaye waapis
teri baaton se bikhar jaaun, haan

maana zamana hai diwaana
isliye tune naa jaana
tere liye main kaafi hun
dekho, dekho, yeh zamane se thak kar

aate ho kyun maasoom ban kar?
tere liye main kya hi hun?

phir aate kyu yahan karne aankhon mei ho baarish?
abh aaye ho thehar jaao naa
aur puchcho naa zara mere din ke baare mei bhi
bas itne mei sambhal jaaun, haan

haan, ek din kabhi koi
jab bhi padhe kahani teri
lagta mujhe mere naam ka
zikr kahin bhi hoga nahi

haan, main yahin
meri yeh aankhon mei, aankhon mei toh dekho
dekho, yeh dil ka haal kya
honthon se hota naa byaan
meri yeh aankhon mei, aankhon mei toh dekho

kaisa naseeb hai mera
milke bhi

---

In [19]:
# language detection
print(ld.detect_langs(lyrics.lower()))

[so:0.7142850086007764, id:0.28571417988026676]
